![nba-logo](images/nba-logo.png)

**Una empresa dedicada a las apuestas, necesita un modelo de entrenamiento fiable en el cual poder saber si gana un equipo u otro dependiendo de unos datos de entrada. En este ejercicio se haran pruebas de distintos modelos y se seleccionaran distintas variables para probar su eficacia.**

In [1]:
import pandas as pd 
import numpy as np
from IPython.display import display
import zipfile as zp
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import sklearn.preprocessing as prep
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import precision_score, recall_score, confusion_matrix, mean_absolute_error, mean_absolute_percentage_error, r2_score, accuracy_score, mean_squared_error

Carga de los dataframes

In [2]:
zf = zp.ZipFile('dataframes/nba.zip')
df_games = pd.read_csv(zf.open('games.csv'))
df_games_details = pd.read_csv(zf.open('games_details.csv'))
df_players = pd.read_csv(zf.open('players.csv'))
df_ranking = pd.read_csv(zf.open('ranking.csv'))
df_teams = pd.read_csv(zf.open('teams.csv'))

/tmp/ipykernel_5567/1756342927.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_games_details = pd.read_csv(zf.open('games_details.csv'))


Muestra varias filas de cada dataframe para realizar un primer analisis

In [3]:
print("--------- PARTIDOS ----------")
print("-----------------------------")
display(df_games.head())
print("--------- DETALLES PARTIDOS ----------")
print("--------------------------------------")
display(df_games_details.head())
print("--------- JUGADORES ----------")
print("------------------------------")
display(df_players.head())
print("--------- RANKING LIGA ----------")
print("---------------------------------")
display(df_ranking.head())
print("--------- EQUIPOS ----------")
print("----------------------------")
display(df_teams.head())

--------- PARTIDOS ----------
-----------------------------


,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2022-12-22,22200477,Final,1610612740,1610612759,2022,1610612740,126.0,0.484,0.926,...,25.0,46.0,1610612759,117.0,0.478,0.815,0.321,23.0,44.0,1
1,2022-12-22,22200478,Final,1610612762,1610612764,2022,1610612762,120.0,0.488,0.952,...,16.0,40.0,1610612764,112.0,0.561,0.765,0.333,20.0,37.0,1
2,2022-12-21,22200466,Final,1610612739,1610612749,2022,1610612739,114.0,0.482,0.786,...,22.0,37.0,1610612749,106.0,0.470,0.682,0.433,20.0,46.0,1
3,2022-12-21,22200467,Final,1610612755,1610612765,2022,1610612755,113.0,0.441,0.909,...,27.0,49.0,1610612765,93.0,0.392,0.735,0.261,15.0,46.0,1
4,2022-12-21,22200468,Final,1610612737,1610612741,2022,1610612737,108.0,0.429,1.000,...,22.0,47.0,1610612741,110.0,0.500,0.773,0.292,20.0,47.0,0


--------- DETALLES PARTIDOS ----------
--------------------------------------


,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,22200477,1610612759,SAS,San Antonio,1629641,Romeo Langford,Romeo,F,NaN,18:06,...,1.0,1.0,2.0,0.0,1.0,0.0,2.0,5.0,2.0,-2.0
1,22200477,1610612759,SAS,San Antonio,1631110,Jeremy Sochan,Jeremy,F,NaN,31:01,...,6.0,3.0,9.0,6.0,1.0,0.0,2.0,1.0,23.0,-14.0
2,22200477,1610612759,SAS,San Antonio,1627751,Jakob Poeltl,Jakob,C,NaN,21:42,...,1.0,3.0,4.0,1.0,1.0,0.0,2.0,4.0,13.0,-4.0
3,22200477,1610612759,SAS,San Antonio,1630170,Devin Vassell,Devin,G,NaN,30:20,...,0.0,9.0,9.0,5.0,3.0,0.0,2.0,1.0,10.0,-18.0
4,22200477,1610612759,SAS,San Antonio,1630200,Tre Jones,Tre,G,NaN,27:44,...,0.0,2.0,2.0,3.0,0.0,0.0,2.0,2.0,19.0,0.0


--------- JUGADORES ----------
------------------------------


,PLAYER_NAME,TEAM_ID,PLAYER_ID,SEASON
0,Royce O'Neale,1610612762,1626220,2019
1,Bojan Bogdanovic,1610612762,202711,2019
2,Rudy Gobert,1610612762,203497,2019
3,Donovan Mitchell,1610612762,1628378,2019
4,Mike Conley,1610612762,201144,2019


--------- RANKING LIGA ----------
---------------------------------


,TEAM_ID,LEAGUE_ID,SEASON_ID,STANDINGSDATE,CONFERENCE,TEAM,G,W,L,W_PCT,HOME_RECORD,ROAD_RECORD,RETURNTOPLAY
0,1610612743,0,22022,2022-12-22,West,Denver,30,19,11,0.633,10-3,9-8,NaN
1,1610612763,0,22022,2022-12-22,West,Memphis,30,19,11,0.633,13-2,6-9,NaN
2,1610612740,0,22022,2022-12-22,West,New Orleans,31,19,12,0.613,13-4,6-8,NaN
3,1610612756,0,22022,2022-12-22,West,Phoenix,32,19,13,0.594,14-4,5-9,NaN
4,1610612746,0,22022,2022-12-22,West,LA Clippers,33,19,14,0.576,11-7,8-7,NaN


--------- EQUIPOS ----------
----------------------------


,LEAGUE_ID,TEAM_ID,MIN_YEAR,MAX_YEAR,ABBREVIATION,NICKNAME,YEARFOUNDED,CITY,ARENA,ARENACAPACITY,OWNER,GENERALMANAGER,HEADCOACH,DLEAGUEAFFILIATION
0,0,1610612737,1949,2019,ATL,Hawks,1949,Atlanta,State Farm Arena,18729.0,Tony Ressler,Travis Schlenk,Lloyd Pierce,Erie Bayhawks
1,0,1610612738,1946,2019,BOS,Celtics,1946,Boston,TD Garden,18624.0,Wyc Grousbeck,Danny Ainge,Brad Stevens,Maine Red Claws
2,0,1610612740,2002,2019,NOP,Pelicans,2002,New Orleans,Smoothie King Center,NaN,Tom Benson,Trajan Langdon,Alvin Gentry,No Affiliate
3,0,1610612741,1966,2019,CHI,Bulls,1966,Chicago,United Center,21711.0,Jerry Reinsdorf,Gar Forman,Jim Boylen,Windy City Bulls
4,0,1610612742,1980,2019,DAL,Mavericks,1980,Dallas,American Airlines Center,19200.0,Mark Cuban,Donnie Nelson,Rick Carlisle,Texas Legends


Muestra filas y columnas de cada dataframe

In [4]:
print("--------- PARTIDOS ----------")
print("-----------------------------")
display(df_games.shape)
print("--------- DETALLES PARTIDOS ----------")
print("--------------------------------------")
display(df_games_details.shape)
print("--------- JUGADORES ----------")
print("------------------------------")
display(df_players.shape)
print("--------- RANKING LIGA ----------")
print("---------------------------------")
display(df_ranking.shape)
print("--------- EQUIPOS ----------")
print("----------------------------")
display(df_teams.shape)

--------- PARTIDOS ----------
-----------------------------


(26651, 21)

--------- DETALLES PARTIDOS ----------
--------------------------------------


(668628, 29)

--------- JUGADORES ----------
------------------------------


(7228, 4)

--------- RANKING LIGA ----------
---------------------------------


(210342, 13)

--------- EQUIPOS ----------
----------------------------


(30, 14)

Vemos la información

In [5]:
print("--------- PARTIDOS ----------")
print("-----------------------------")
display(df_games.dtypes)
print("--------- DETALLES PARTIDOS ----------")
print("--------------------------------------")
display(df_games_details.dtypes)
print("--------- JUGADORES ----------")
print("------------------------------")
display(df_players.dtypes)
print("--------- RANKING LIGA ----------")
print("---------------------------------")
display(df_ranking.dtypes)
print("--------- EQUIPOS ----------")
print("----------------------------")
display(df_teams.dtypes)

--------- PARTIDOS ----------
-----------------------------


GAME_DATE_EST        object
GAME_ID               int64
GAME_STATUS_TEXT     object
HOME_TEAM_ID          int64
VISITOR_TEAM_ID       int64
SEASON                int64
TEAM_ID_home          int64
PTS_home            float64
FG_PCT_home         float64
FT_PCT_home         float64
FG3_PCT_home        float64
AST_home            float64
REB_home            float64
TEAM_ID_away          int64
PTS_away            float64
FG_PCT_away         float64
FT_PCT_away         float64
FG3_PCT_away        float64
AST_away            float64
REB_away            float64
HOME_TEAM_WINS        int64
dtype: object

--------- DETALLES PARTIDOS ----------
--------------------------------------


GAME_ID                int64
TEAM_ID                int64
TEAM_ABBREVIATION     object
TEAM_CITY             object
PLAYER_ID              int64
PLAYER_NAME           object
NICKNAME              object
START_POSITION        object
COMMENT               object
MIN                   object
FGM                  float64
FGA                  float64
FG_PCT               float64
FG3M                 float64
FG3A                 float64
FG3_PCT              float64
FTM                  float64
FTA                  float64
FT_PCT               float64
OREB                 float64
DREB                 float64
REB                  float64
AST                  float64
STL                  float64
BLK                  float64
TO                   float64
PF                   float64
PTS                  float64
PLUS_MINUS           float64
dtype: object

--------- JUGADORES ----------
------------------------------


PLAYER_NAME    object
TEAM_ID         int64
PLAYER_ID       int64
SEASON          int64
dtype: object

--------- RANKING LIGA ----------
---------------------------------


TEAM_ID            int64
LEAGUE_ID          int64
SEASON_ID          int64
STANDINGSDATE     object
CONFERENCE        object
TEAM              object
G                  int64
W                  int64
L                  int64
W_PCT            float64
HOME_RECORD       object
ROAD_RECORD       object
RETURNTOPLAY     float64
dtype: object

--------- EQUIPOS ----------
----------------------------


LEAGUE_ID               int64
TEAM_ID                 int64
MIN_YEAR                int64
MAX_YEAR                int64
ABBREVIATION           object
NICKNAME               object
YEARFOUNDED             int64
CITY                   object
ARENA                  object
ARENACAPACITY         float64
OWNER                  object
GENERALMANAGER         object
HEADCOACH              object
DLEAGUEAFFILIATION     object
dtype: object

Ver si hay nulos en los distintos dataframes

In [6]:
print("--------- PARTIDOS ----------")
print("-----------------------------")
display(df_games.isnull().sum())
display(df_games[df_games.isna().any(axis=1)])

--------- PARTIDOS ----------
-----------------------------


GAME_DATE_EST        0
GAME_ID              0
GAME_STATUS_TEXT     0
HOME_TEAM_ID         0
VISITOR_TEAM_ID      0
SEASON               0
TEAM_ID_home         0
PTS_home            99
FG_PCT_home         99
FT_PCT_home         99
FG3_PCT_home        99
AST_home            99
REB_home            99
TEAM_ID_away         0
PTS_away            99
FG_PCT_away         99
FT_PCT_away         99
FG3_PCT_away        99
AST_away            99
REB_away            99
HOME_TEAM_WINS       0
dtype: int64

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
19175,2003-10-24,10300116,Final,1610612753,1610612762,2003,1610612753,NaN,NaN,NaN,...,NaN,NaN,1610612762,NaN,NaN,NaN,NaN,NaN,NaN,0
19176,2003-10-24,10300108,Final,1610612737,1610612764,2003,1610612737,NaN,NaN,NaN,...,NaN,NaN,1610612764,NaN,NaN,NaN,NaN,NaN,NaN,0
19177,2003-10-24,10300109,Final,1610612738,1610612751,2003,1610612738,NaN,NaN,NaN,...,NaN,NaN,1610612751,NaN,NaN,NaN,NaN,NaN,NaN,0
19178,2003-10-24,10300113,Final,1610612759,1610612745,2003,1610612759,NaN,NaN,NaN,...,NaN,NaN,1610612745,NaN,NaN,NaN,NaN,NaN,NaN,0
19179,2003-10-24,10300112,Final,1610612749,1610612765,2003,1610612749,NaN,NaN,NaN,...,NaN,NaN,1610612765,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19269,2003-10-09,10300019,Final,1610612743,1610612756,2003,1610612743,NaN,NaN,NaN,...,NaN,NaN,1610612756,NaN,NaN,NaN,NaN,NaN,NaN,0
19270,2003-10-09,10300022,Final,1610612757,1610612758,2003,1610612757,NaN,NaN,NaN,...,NaN,NaN,1610612758,NaN,NaN,NaN,NaN,NaN,NaN,0
19271,2003-10-08,10300013,Final,1610612759,1610612763,2003,1610612759,NaN,NaN,NaN,...,NaN,NaN,1610612763,NaN,NaN,NaN,NaN,NaN,NaN,0
19278,2003-10-08,10300015,Final,1610612747,1610612744,2003,1610612747,NaN,NaN,NaN,...,NaN,NaN,1610612744,NaN,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
print("--------- DETALLES PARTIDOS ----------")
print("--------------------------------------")
display(df_games_details.isnull().sum())
display(df_games_details[df_games_details.isna().any(axis=1)])

--------- DETALLES PARTIDOS ----------
--------------------------------------


GAME_ID                   0
TEAM_ID                   0
TEAM_ABBREVIATION         0
TEAM_CITY                 0
PLAYER_ID                 0
PLAYER_NAME               0
NICKNAME             615591
START_POSITION       412863
COMMENT              558939
MIN                  109690
FGM                  109690
FGA                  109690
FG_PCT               109690
FG3M                 109690
FG3A                 109690
FG3_PCT              109690
FTM                  109690
FTA                  109690
FT_PCT               109690
OREB                 109690
DREB                 109690
REB                  109690
AST                  109690
STL                  109690
BLK                  109690
TO                   109690
PF                   109690
PTS                  109690
PLUS_MINUS           133351
dtype: int64

In [ ]:
print("--------- JUGADORES ----------")
print("------------------------------")
display(df_players.isnull().sum())
display(df_players[df_players.isna().any(axis=1)])

In [ ]:
print("--------- RANKING LIGA ----------")
print("---------------------------------")
display(df_ranking.isnull().sum())
display(df_ranking[df_ranking.isna().any(axis=1)])

In [ ]:
print("--------- EQUIPOS ----------")
print("----------------------------")
display(df_teams.isnull().sum())
display(df_teams[df_teams.isna().any(axis=1)])

En el df de partidos podemos ver que hay NaN en el 2003, borramos todas las filas con partidos anteriores al 2010 y comprobamos si se han quedado NaN

In [ ]:
df_games.isna().sum()[df_games.isna().sum()>0].plot(kind='bar')

In [ ]:
df_games = df_games.loc[df_games['GAME_DATE_EST'] >= "2004-01-01"].reset_index(drop=True)

In [ ]:
df_games.isnull().sum()

Para saber que equipo es, podríamos fusionar algunos datos del dataframe de partidos con el de equipos de alguna manera

In [ ]:
display(df_games.columns)
display(df_teams.columns)

Vemos como podemos sustituir los IDS por los nombres de los equipos

In [ ]:
df_teams = df_teams[['TEAM_ID', 'NICKNAME']]

# Reemplaza HOME_TEAM_ID por los nombres del dataframe teams
nombres_local = df_teams.copy()
nombres_local.columns = ['HOME_TEAM_ID', 'NICKNAME']
# Se unen el ID de lequipo por el nickname
result_1 = pd.merge(df_games['HOME_TEAM_ID'], nombres_local, how ="left", on="HOME_TEAM_ID")  
df_games['HOME_TEAM_ID'] = result_1['NICKNAME']


Vemos como se ha cambiado el HOME_TEAM_ID por el nombre del equipo, haremos lo mismo con el visitante

In [ ]:
df_games.head(20)

Haremos lo mismo con los equipos visitantes

In [ ]:
# Reemplaza VISITOR_TEAM_ID por los nombres del dataframe teams
nombres_visitante = df_teams.copy()
nombres_visitante.columns = ['VISITOR_TEAM_ID', 'NICKNAME']
# Se unen el ID del equipo por el nickname
result_2 = pd.merge(df_games['VISITOR_TEAM_ID'], nombres_visitante, how ="left", on="VISITOR_TEAM_ID")  
df_games['VISITOR_TEAM_ID'] = result_2['NICKNAME']

In [ ]:
df_games.head()

Exploración de los valores únicos de las variables del dataframe de partidos

In [ ]:

for column in df_games.columns:
    if len(df_games[column].unique()) < 10:
        print(column, df_games[column].unique())
    else:
        continue

Al menos GAME_STATUS_TEXT nos sobra ya que solo tiene un valor unico y no tiene relevancia

In [ ]:
df_games = df_games.drop(columns=['GAME_STATUS_TEXT'])
df_games.head()

Selecciono solo la temporada 2022

In [ ]:
df_games_2022 = df_games.loc[df_games['SEASON'] == 2022]

Vemos cantidad de filas y columnas del nuevo dataframe seleccionando 2022

In [ ]:
df_games_2022.shape

Guardamos las columnas en una variable llamada variables

In [ ]:
variables = list(df_games_2022.columns)
print(variables)

Guardamos las variables numericas en una lista y luego creamos otra lista con las mismas variables pero añadiendo norm para saber que están normalizadas

In [ ]:
variables_elegidas = ['PTS_home', 'FG_PCT_home', 'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home', 'PTS_away', 'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away', 'REB_away']
variables_elegidas_norm = ['PTS_home_norm', 'FG_PCT_home_norm', 'FT_PCT_home_norm', 'FG3_PCT_home_norm', 'AST_home_norm', 'REB_home_norm', 'PTS_away_norm', 'FG_PCT_away_norm', 'FT_PCT_away_norm', 'FG3_PCT_away_norm', 'AST_away_norm', 'REB_away_norm']


Añadir al dataframe "X" las variables que hemos elegido para entrenar y al "y" el target, esto dirá si gana el equipo local 0 y si gana el visitante 1

In [ ]:
X = df_games_2022[variables_elegidas]
y = df_games_2022["HOME_TEAM_WINS"]

Muestra el conjunto de datos de la temporada 2022 y el target

In [ ]:
display(X.head())
display(y.head())

Hacemos el split de train y de test, 0.7 y 0.3 respectivamente

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.3, random_state=42)

Normalizamos las variables elegidas en train

In [ ]:
scaler = prep.MinMaxScaler()
X = scaler.fit_transform(X_train[variables_elegidas])

# Creamos columnas con datos normalizados
X_train[variables_elegidas_norm] = X
# Borramos las variables no normalizadas
X_train = X_train.select_dtypes(include = 'number').drop(variables_elegidas, axis = 1)

In [ ]:
X_train.head()

Normalizamos test

In [ ]:
X = scaler.transform(X_test[variables_elegidas])

# Creamos columnas con datos normalizados
X_test[variables_elegidas_norm] = X
# Borramos las variables no normalizadas
X_test = X_test.select_dtypes(include = 'number').drop(variables_elegidas, axis = 1)

Vemos cuales son las mejores 6 variables correlacionadas con y, también la correlación de todas

In [ ]:
selector = SelectKBest(chi2, k=6)

X_select = selector.fit_transform(X_train, y_train)
print(selector.get_feature_names_out())
for var, value in zip(selector.feature_names_in_, selector.scores_):
    print('Variable %s: %.4f' % (var, value))

APLICAMOS EL ENTRENAMIENTO DE MODELOS

Empezamos usando regresión logistíca

In [ ]:
# Creamos el objeto del modelo con parámetros por defecto, fijando la semilla para evitar aleatoriedad
logreg = LogisticRegression(random_state=42)

# Entrenamos con el conjunto de train y su target
logreg.fit(X_train, y_train)
print('Accuracy en train: ', logreg.score(X_train, y_train))
print('Accuracy en test: ', logreg.score(X_test, y_test))

Usamos el algoritmo de maquinas de vector soporte de clasificación con una pipeline de scikit-learn

In [ ]:
pipe = Pipeline([('scaler', StandardScaler()), ('svc', SVC())])
pipe.fit(X_train, y_train)

pipe.score(X_test, y_test)